In [ ]:
import sunpy.map
from astropy.io import fits
#import nustar_pysolar as nustar
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord, SkyOffsetFrame

# path_to_dodem = '/Users/jmdunca2/do-dem/'
# from sys import path as sys_path
# sys_path.append(path_to_dodem+'/dodem/')
# import nustar_dem_prep as nu

For some observations, a NuSTAR source at the limb does not have an obvious corresponding source in AIA channels that would be expected to have overlapping temperature sensitivity (particularly we look at 94 and 131 \AA). Here we check on the AIA/NuSTAR observer coordinates to partially determine how much of a difference in the limb location there could be, due to differences in the locations of the two spacecraft. 

AIA is in a geosynchronous orbit. It is \~35,350 km from the earth surface (\~41,7500 km from earth center)
NuSTAR is in a near-equitorial low earth orbit, meaning it is at most 600 km from the earth surface. (\~7000 km from earth center). This declines over the course of the instrument lifetime by some %. 

For each AIA file, we can get the observer coordinate (where AIA is located) accurately by creating a sunpy map (and using the observer_coordinate attribute). 

For NuSTAR, we don't have accurate observer information in the same location. 

In [ ]:
#February 19, 2016 case:

evt_file = '/Users/jmdunca2/do-dem/initial_dem_19feb16/18-54-50_19-08-55/nu20102011001A06_0_4_p_cl_sunpos.evt'


with fits.open(evt_file) as hdu:
    evt_data = hdu[1].data
    hdr = hdu[1].header

nustar_map = nustar.map.make_sunpy(evt_data, hdr)


aia_file = './aia.lev1.94A_2016-02-19T185400.12Z.image_lev1.fits'
aia_map = sunpy.map.Map(aia_file)

print(aia_map.observer_coordinate.lon.to(u.arcsec))
print(aia_map.observer_coordinate.cartesian.xyz.to(u.km))
print(aia_map.observer_coordinate.cartesian.y.to(u.km)/(750*u.km/u.arcsec))

The above coordinates are in cartesian coordinates centered on the Sun (HEEQ)
As defined in: https://www.mssl.ucl.ac.uk/grid/iau/extra/local_copy/SP_coords/heliosys.htm#:~:text=1.1%20Heliocentric%20Earth%20equatorial%20(HEEQ,as%20heliocentric%20solar%20(HS).

- The x is basically 1 AU (measured along "the intersection of the solar equator and the solar central meridian as seen from the Earth")
- The z is parallel to the Sun's rotation axis. 
- The y is perpendicular to both the above, so like essentially solar east-west.

We see that when we convert the y to arcseconds on the sun, it's about the same as the longitude. 

In [ ]:
print(nustar_map.observer_coordinate)
print(nustar_map.observer_coordinate.cartesian)

The two zeros (longitude, HEEQ y) indicate that the observer coordinate is just set to the center of the earth for NuSTAR. 

In order to get the actual nustar position, we need to retrieve a tle file which has the information needed to make an EarthSatellite object (using skyfield). We then can turn it into a skyfield object and get it into the same units as the above: 

In [ ]:
import nustar_pysolar.io as io
from skyfield.api import EarthSatellite
import datetime

tlefile=io.download_tle(outdir='/Users/jmdunca2/do-dem/initial_dem_19feb16/')
mindt, line1, line2 = io.get_epoch_tle(datetime.datetime(2016, 2, 19, 18, 54, 55), tlefile)
nustar_sat = EarthSatellite(line1, line2)
print('')
print(nustar_sat.at(nustar_sat.epoch).to_skycoord().heliographic_stonyhurst.cartesian.xyz.to(u.km))
print(nustar_sat.at(nustar_sat.epoch).to_skycoord().heliographic_stonyhurst.cartesian.y.to(u.km)/(750*u.km/u.arcsec))

Thus, we see that the observers are on opposite sides of the earth-Sun line, and interestingly that NuSTAR is about as far from the earth-sun line as it will ever get (\~6500 km, when its orbit is \~7000 km in radius. Is this true for the other cases as well?

In [ ]:
aia_file = './aia.lev1.94A_2016-04-22T172000.12Z.image_lev1.fits'

aia_map = sunpy.map.Map(aia_file)

print('AIA Longitude: ', aia_map.observer_coordinate.lon.to(u.arcsec))
print('AIA HEEQ Cartesian: ', aia_map.observer_coordinate.cartesian.xyz.to(u.km))
print('HEEQ y to arcsec: ', aia_map.observer_coordinate.cartesian.y.to(u.km)/(750*u.km/u.arcsec))

tlefile=io.download_tle(outdir='/Users/jmdunca2/do-dem/initial_dem_22apr16_2/')
mindt, line1, line2 = io.get_epoch_tle(datetime.datetime(2016, 4, 22, 17, 20), tlefile)
nustar_sat = EarthSatellite(line1, line2)
print('')
print('NuSTAR HEEQ Cartesian: ', nustar_sat.at(nustar_sat.epoch).to_skycoord().heliographic_stonyhurst.cartesian.xyz.to(u.km))
print('HEEQ y to arcsec: ', nustar_sat.at(nustar_sat.epoch).to_skycoord().heliographic_stonyhurst.cartesian.y.to(u.km)/(750*u.km/u.arcsec))

In [ ]:
aia_file = './aia.lev1.94A_2018-05-29T211611.12Z.image_lev1.fits'

aia_map = sunpy.map.Map(aia_file)

print('AIA Longitude: ', aia_map.observer_coordinate.lon.to(u.arcsec))
print('AIA HEEQ Cartesian: ', aia_map.observer_coordinate.cartesian.xyz.to(u.km))
print('HEEQ y to arcsec: ', aia_map.observer_coordinate.cartesian.y.to(u.km)/(750*u.km/u.arcsec))

tlefile=io.download_tle(outdir='/Users/jmdunca2/do-dem/initial_dem_29may18_2/')
mindt, line1, line2 = io.get_epoch_tle(datetime.datetime(2018, 5, 29, 21, 16, 11), tlefile)
nustar_sat = EarthSatellite(line1, line2)
print('')
print('NuSTAR HEEQ Cartesian: ', nustar_sat.at(nustar_sat.epoch).to_skycoord().heliographic_stonyhurst.cartesian.xyz.to(u.km))
print('HEEQ y to arcsec: ', nustar_sat.at(nustar_sat.epoch).to_skycoord().heliographic_stonyhurst.cartesian.y.to(u.km)/(750*u.km/u.arcsec))

For both 2016 February 19 and 2018 May 29, NuSTAR is on the "solar west" side of the earth-sun line (and AIA is on the other side). This could support an interpretation in which NuSTAR can see farther over the limb than AIA, based on the orbital geometry. However, for the third case (2016 April 22), the opposite is true: AIA is quite far to the "solar west" of NuSTAR. 

Next step: to evaluate further, it would be interesting to do some further geometry, such as:

- determine the approximate angular position of the active region most likely to be the NuSTAR source in each case
- determine the typical height of coronal loops in that/any region
- determine whether such an altitude is consistent with a source being in NuSTAR's line of sight but not AIA's for each of the cases with the "correct" geometry.

It would also be good to do spectroscopy to evaluate the potential validity of a purely non-thermal interpretation of the NuSTAR emission (unlikely, but very interesting). 

In [ ]:
# fig = plt.figure(figsize=(14,22))


# #Set broad box for plotting (using region object)
# bl=[(0)*u.arcsec, (-500)*u.arcsec]
# tr=[(2000)*u.arcsec,(1000)*u.arcsec]
# #print(tr[0]-bl[0], tr[1]-bl[1])

# m = aia_map

# bottom_left = SkyCoord(bl[0]-100*u.arcsec, bl[1]-100*u.arcsec, frame=m.coordinate_frame)
# top_right = SkyCoord(tr[0]+100*u.arcsec,tr[1]+100*u.arcsec, frame=m.coordinate_frame)
# mm = m.submap(bottom_left=bottom_left, top_right=top_right)
# mm.peek()

# fpm='A'
# datapath='./' #not using, as input specific evt
# nu_smap = nu.return_submap(datapath=datapath, fpm=fpm, specific_evt=evt_file, bl=bl, tr=tr, already_sunpos=True)
# nu_smap.peek()

In [ ]:
aia_file = './aia.lev1.94A_2021-01-20T134610.62Z.image_lev1.fits'

mm = sunpy.map.Map(aia_file)
mm.fits_header['QUALITY']

In [ ]:
mm.fits_header['AISTATE']

In [ ]:
mm.fits_header['ACS_MODE']

In [ ]:
import aiapy.util
print(aiapy.util.check_quality_flag(mm.fits_header['QUALITY']))

In [ ]:
import sunpy.map
import aiapy.psf
aia_file = './aia.lev1.94A_2016-02-19T185400.12Z.image_lev1.fits'
aia_map = sunpy.map.Map(aia_file)

In [ ]:
aia_map_deconvolved = aiapy.psf.deconvolve(aia_map)

In [ ]:
aia_map

In [ ]:
from astropy.coordinates import SkyCoord, SkyOffsetFrame
from astropy import units as u
from matplotlib import pyplot as plt
from astropy.visualization import AsinhStretch, ImageNormalize

left_corner = 0 * u.arcsec, 0 * u.arcsec
right_corner = 700 * u.arcsec, 500 * u.arcsec

aia_map_sub = aia_map.submap(
    bottom_left=SkyCoord(*left_corner, frame=aia_map.coordinate_frame),
    top_right=SkyCoord(*right_corner, frame=aia_map.coordinate_frame),
)
aia_map_deconvolved_sub = aia_map_deconvolved.submap(
    bottom_left=SkyCoord(*left_corner, frame=aia_map_deconvolved.coordinate_frame),
    top_right=SkyCoord(*right_corner, frame=aia_map_deconvolved.coordinate_frame),
)

fig = plt.figure()

norm = ImageNormalize(vmin=0, vmax=1.5e2, stretch=AsinhStretch(0.01))

ax = fig.add_subplot(121, projection=aia_map_sub)
aia_map_sub.plot(axes=ax, norm=norm)
ax.set_title("Normal")

ax = fig.add_subplot(122, projection=aia_map_deconvolved_sub)
aia_map_deconvolved_sub.plot(axes=ax, annotate=False, norm=norm)
ax.set_title("Deconvolved")
ax.coords[0].set_axislabel(" ")
ax.coords[1].set_axislabel(" ")
ax.coords[1].set_ticklabel_visible(visible=False)

plt.show()

In [ ]:
import numpy as np
print(np.min(aia_map.data), np.max(aia_map.data), np.sum(aia_map.data))
print(np.min(aia_map_deconvolved.data), np.max(aia_map_deconvolved.data), np.sum(aia_map_deconvolved.data))
print(np.sum(aia_map.data)/np.sum(aia_map_deconvolved.data))